# JESS API Example
This notebook shows a simple example of running optimisation using the [JESS Engine API](https://www.jeplus.org/wiki/doku.php?id=docs:jea:jea_api "Go to JESS API docs"). 

Let's start with loading the necessary package and testing the connection to the server. The "[Requests: HTTP for Humans](http://python-requests.org "Go to Python Requests website")" is shown here, although other http packaages can be used too. 

After loading the Request package, we can run the Info command. Info is the simplest command and does not require authentication, so it works also if you copy and paste the URL to a browser, or [click here](https://api.ensims.com/jess_web/api/info "Try Info command in browser").

In [53]:
# Use requests (see https://requests.readthedocs.io/en/master/)
import requests

# API endpoint URL
ApiBase = 'https://api.ensims.com/'
# ApiBase = 'https://localhost:8443/'
JessApi = ApiBase + "jess_web/api/"
UserApi = ApiBase + 'users/api/'

# Test connection
r = requests.get(JessApi + 'info')
r.json()

{'Title': 'JESS Web API',
 'Description': 'JESS online simulation services API provided by ENSIMS Ltd.',
 'Major': 1,
 'Minor': 0,
 'Revision': 0,
 'Release': 'beta',
 'Update': 1,
 'Notice': '(C) 2020, Energy Simulation Solutions Ltd. All rights reserved.'}

The request returns data in JSON format, which most of the JESS Web API uses. To see the data, use the 'json()' function. We can also access its fields like this:

In [2]:
r.json()['Title']

'JESS Web API'

## Authentication

First, we need to log on to the JESS Web platform. More details of the authentication and account commands are available in the Users_API example notebook and the releveant documentation. Here, we just log on using the existing credential and then store the cookies.

In [54]:
# Get log in credential
import getpass
user_email = "yi@jeplus.org"
password = getpass.getpass("Password for " + user_email + ": ")

# Set header and body of the POST request
headers = {'Content-Type': 'application/json'}
body = {"email": user_email, "password": password}

# Send request
r = requests.post(UserApi + 'auth', headers=headers, json=body)

# Keep the cookies which contain the session token for secured requests
cookies = r.cookies

# Show data returned by JESS
r.json()

Password for yi@jeplus.org:  ········


{'ok': True,
 'status': 'Logged in successfully!',
 'jwt': 'Session token in cookie',
 'user': 'Yi',
 'role': 'pro',
 'email': 'yi@jeplus.org'}

**If you have already done the step described below, please move on to the Run Simulations section**

If this is the first time you access JESS in this way, you will enable the service by linking an existing JESS account to your ENSIMS web service account. Please note that JESS is paid service and its accounts are provided on request only. Please use this form to make a request: [http://cms.ensims.com/index.php/jess-online/get-jess-account]

This is how to enable JESS:

In [57]:
# JESS account creds 
username = "TestUser"
password = getpass.getpass("JESS account password for " + username + ": ")

# Set header and body of the POST request
headers = {'Content-Type': 'application/json'}
body = {"jessUser": username, "jessPwd": password}

# Send request
r = requests.post(JessApi + 'session', headers=headers, json=body, cookies=cookies)

# Show returned message
r.json()

JESS account password for TestUser:  ········


{'ok': True, 'status': 'New JESS session key is assigned.', 'data': None}

## Run simulations directly

The quickest way to run simulations is by using the upload and run command (POST job/). This involves wrapping all the information about the simulation job to run, including the necessary files, in multipart form data. Here is an example.

In [11]:
pwd

'C:\\dev\\python\\JESS-Web-API'

In [69]:
# upload a file to a particular folder. Becareful that the file name fields and the model/weather fields must match!
files = [
    ('file', ('5ZoneAirCooled-v93.idf', open('job_example\\5ZoneAirCooled-v93.idf', 'rb'), 'text/plain')),
    ('file', ('in.epw', open('job_example\\in.epw', 'rb'), 'text/plain')),
    ('title', 'Test job'),
    ('desc', 'This is test submission made from python example'),
    ('model', '5ZoneAirCooled-v93.idf'),
    ('split', 'FALSE'),
    ('weather', 'in.epw')
]

# POST with files
r = requests.post(JessApi + 'job', files=files, cookies=cookies)

# Get job_id. This id number will be needed for querying and retrieving the job data
job_id = r.json()['data']

# Show the returned status
r.json()

{'ok': True, 'status': 'Job 303264 is created', 'data': 303264}

If job submission is successful, JESS returns an acknowledgement with the ok status set to True, and the job id in the 'data' field. The job id is important for accessing simulation later.

## Check job status



In [72]:
# GET job status with job_id
r = requests.get(JessApi + 'job/status/' + str(job_id), cookies=cookies)

# Show the returned status
r.json()

{'ok': True,
 'status': '303264 status available',
 'data': {'completion_Time': 1600773747018,
  'cancel_Flag': False,
  'last_Update': 1600773747029,
  'status_Info': 'Job finished. Results collected into tables.',
  'job_ID': 303264,
  'progress': '1, 0, 0, 0, 0, 0, 1',
  'id': 303209,
  'status': 'FINISHED'}}

## Retrieve job output



In [73]:
# GET job status with job_id
r = requests.get(JessApi + 'job/dir/' + str(job_id), cookies=cookies)

# Show the returned status
r.json()

{'ok': True,
 'status': '303264 status available',
 'data': {'fileName': 'Result folder',
  'open': True,
  'nocheck': True,
  'fileSize': 0,
  'children': [{'fileName': 'console.log',
    'open': False,
    'nocheck': False,
    'fileSize': 5430,
    'children': [],
    'name': 'console.log (5.43 KB)'},
   {'fileName': 'eplusout.bnd',
    'open': False,
    'nocheck': False,
    'fileSize': 62315,
    'children': [],
    'name': 'eplusout.bnd (62.31 KB)'},
   {'fileName': 'eplusout.dxf',
    'open': False,
    'nocheck': False,
    'fileSize': 17064,
    'children': [],
    'name': 'eplusout.dxf (17.06 KB)'},
   {'fileName': 'eplusout.eio',
    'open': False,
    'nocheck': False,
    'fileSize': 54891,
    'children': [],
    'name': 'eplusout.eio (54.89 KB)'},
   {'fileName': 'eplusout.end',
    'open': False,
    'nocheck': False,
    'fileSize': 97,
    'children': [],
    'name': 'eplusout.end (0.10 KB)'},
   {'fileName': 'eplusout.err',
    'open': False,
    'nocheck': False,
 

Okay, now we can retrieve any file in its text form from the server:

In [65]:
# GET specific job output with job_id and file name
r = requests.get(JessApi + 'job/file/' + str(job_id) + "/eplusout.err", cookies=cookies)

# Show the returned status
print(r.text)

Program Version,EnergyPlus, Version 9.3.0-baff08990c, YMD=2020.09.22 12:17,
   ************* Beginning Zone Sizing Calculations
   ** Warning ** Weather file location will be used rather than entered (IDF) Location object.
   **   ~~~   ** ..Location object=CHICAGO_IL_USA TMY2-94846
   **   ~~~   ** ..Weather File Location=LONDON/GATWICK - GBR IWEC Data WMO#=037760
   **   ~~~   ** ..due to location differences, Latitude difference=[9.37] degrees, Longitude difference=[87.57] degrees.
   **   ~~~   ** ..Time Zone difference=[6.0] hour(s), Elevation difference=[67.37] percent, [128.00] meters.
   ************* Beginning System Sizing Calculations
   ************* Beginning Plant Sizing Calculations
   ************* Testing Individual Branch Integrity
   ************* All Branches passed integrity testing
   ************* Testing Individual Supply Air Path Integrity
   ************* All Supply Air Paths passed integrity testing
   ************* Testing Individual Return Air Path Integrit

Or download all files of the job as a zip package, and write it to the local disk.

In [25]:
# Download all job output in as a zip file
r = requests.get(JessApi + 'job/file/' + str(job_id), cookies=cookies)

# Show the returned status
bytes = open('job_example\\' + str(job_id) + '.zip', 'wb').write(r.content)

# Show the returned status
print(str(bytes) + ' bytes written to ' + 'job_example\\' + str(job_id) + '.zip')

3129240 bytes written to job_example\303243.zip


## File viewers

So this is how we can run simulations on JESS and then retrieve the results. But this is not all what the new JESS web service allows us to do. Here is an example of how to the file viewer.

First, lets see what jobs we have run on JESS today:

In [74]:
# !! This part is not working !!
from datetime import date

today = date.today()

# YYYY-mm-dd
dstr = today.strftime("%Y-%m-%d")
print(dstr)

# GET the list of jobs fit the given criteria
filter = {"since": dstr, "till": dstr, "status": "FINISHED"}

r = requests.get(JessApi + 'jobs', headers=headers, json=filter, cookies=cookies)

r.json()

2020-09-22


{'ok': True,
 'status': 'TestUser jobs available',
 'data': [{'cputime': 6,
   'job_SN': 303263,
   'job_Subset': 'ALL',
   'job_Args': '',
   'received': 1600773456093,
   'njobs': 1,
   'submissionSize': 0,
   'storageSize': 15054912,
   'resultSize': 0,
   'job_Name': 'Test job',
   'progress': '1, 0, 0, 0, 0, 0, 1',
   'id': 1,
   'status': 'FINISHED',
   'username': 'TestUser',
   'job_Type': 'EP_IDF',
   'message': 'Job finished. Results collected into tables.'},
  {'cputime': 6,
   'job_SN': 303264,
   'job_Subset': 'ALL',
   'job_Args': '',
   'received': 1600773727690,
   'njobs': 1,
   'submissionSize': 0,
   'storageSize': 15072039,
   'resultSize': 0,
   'job_Name': 'Test job',
   'progress': '1, 0, 0, 0, 0, 0, 1',
   'id': 1,
   'status': 'FINISHED',
   'username': 'TestUser',
   'job_Type': 'EP_IDF',
   'message': 'Job finished. Results collected into tables.'}]}

Now we can show a DXF model viewer here using an IFrame.

In [81]:
from IPython.display import IFrame    
IFrame(src='https://app.ensims.com/3d.html?model=' + str(job_id) + '/eplusout.dxf', width='100%', height=650)

It is also possible to access the 3D model in JSON format, which may be useful for integrating with other renders.

In [75]:
# Or to request the DXF to be processed and a model reference returned
r = requests.get(JessApi + 'dxf/' + str(job_id) + '/eplusout.dxf', cookies=cookies)

# Save model ref for accessing json 3D model data
model_ref = r.json()['data']

r.json()

{'ok': True,
 'status': 'Model is ready to view. Use the model reference in the data field.',
 'data': 'bb958233c3ff59d139ccd1dd48d8a733'}

This is the end of the current demo. More demos are in development and will be released, soon.

## End of Demo

This is the end of the current demo. More demos are in development and will be released, soon.
